In [ ]:
import trfp
import pandas as pd
import numpy as np
import scipy

import matplotlib.pyplot as plt

def multipole(order, skew, strength, x_pos, y_pos):
    """Returns the magnitude of a B-field given multipole parameters
    and position.
    """
    # Takes a multipole strength A normalized to 4.5 cm
    r_pos = np.sqrt(x_pos**2 + y_pos**2)
    theta = np.arctan2(y_pos, x_pos)
    if skew == 0:
        b_magnitude = strength * (r_pos/4.5)**order * np.cos(order*theta)
    if skew == 1:
        b_magnitude = strength * (r_pos/4.5)**order * np.sin(order*theta)
    return b_magnitude

In [ ]:
print multipole(4, 0, 1, trfp.TR_X, trfp.TR_Y)

print multipole(8, 0, 1, trfp.TR_X, trfp.TR_Y)

In [ ]:
Bs_df = pd.read_csv(filepath_or_buffer='sample_trolley_measurements.csv', index_col=0)
Bs_df.head()

In [ ]:
mults_df = pd.DataFrame(index=Bs_df.index)
trp = ['tr'+str(i) for i in range(17)]
trm = ['tr,'+str(m+1) for m in range(17)]
for m in range(17):
    mults_df[trm[m]] = Bs_df.dot(trfp.THETA_TR[m])
mults_df.head()

In [ ]:
joes_freqs = np.array([52.74127,52.71351,52.76751,52.78379,52.69151,52.60777,52.38269,52.6757,52.8073,52.98435,
                       52.9413,52.9126,52.51062,52.43007,52.64167,52.97253,52.87799])

joes_mults = np.dot(trfp.THETA_TR, joes_freqs)

for ii in np.round(joes_mults,5):
    print ii

In [ ]:
def multipole_fit()

In [ ]:
np.mean(joes_freqs)

In [ ]:
THETA_TR = np.linalg.pinv(np.transpose(np.array([multipole(trfp.matrices.__MULTIPOLE_ORDER[i],
                                                             trfp.matrices.__MULTIPOLE_SKEW[i],
                                                             1, trfp.TR_X,
                                                             trfp.TR_Y
                                                            )
                                                 for i in np.arange(10)])))

joes_mults_2 = np.dot(THETA_TR, joes_freqs)

for ii in np.roung(joes_mults_2, 5):
    print ii

In [ ]:
print THETA_TR

In [ ]:
mult_high = [56000,  # dipole
             2000,  # nq
             2000,  # sq
             2000,  # ss
             1000,  # ns
             1000,  # no
             1500,  # so
             4000,  # nd
             500,  # sd
             3000,  # n12
             5000,  # s12
             500,  # n14
             5000,  # s14
             5000,  # n16
             5000,  # s16
             5000,  # n18
             300]#,  # s18
#              5000,  # n20
#              5000,  # s20
#              5000]  # n22

mult_low = [48000,  # dipole
            -2000,  # nq
            -2000,  # sq
            -2000,  # ss
            -1000,  # ns
            -1000,  # no
            -1500,  # so
            -4000,  # nd
            -500,  # sd
            -3000,  # n12
            -5000,  # s12
            -500,  # n14
            -5000,  # s14
            -5000,  # n16
            -5000,  # s16
            -5000,  # n18
            -300]#,  # s18
#             -5000,  # n20
#             -5000,  # s20
#             -5000]  # n22

mult_high = np.array(mult_high)
mult_low = np.array(mult_low)

mult_avg = (mult_high+mult_low)/2
mult_spread = mult_high - mult_avg

# generate a bunch of random multipole strengths

N = 1000
nParams = 17

truths = np.zeros((N, mult_high.size))

for ii in range(N):
    for mult in range(mult_high.size):
        truths[ii, mult] = np.random.uniform(mult_avg[mult], mult_spread[mult])
        
# for each row of truths, calculate fields at each probe, then fit those

mult_order = [0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12]
mult_skew = [0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]

fits = np.zeros((N,nParams))
B = np.zeros((N,17))

for N in range(truths.shape[0]):
    for ii in range(truths[N].size):
        B[N,:] += multipole(mult_order[ii], mult_skew[ii], truths[N, ii], trfp.TR_X, trfp.TR_Y)
    fit_mults = np.matmul(trfp.THETA_TR, B[N,:])
    fits[N, :] = fit_mults
    
errors = truths[:,0:17]-fits

In [ ]:
plt.plot(errors[:,7],'.')